## Práctica MongoDB

### Configuración incial

In [2]:
import json
from pymongo import MongoClient
from datetime import datetime

from collections import Counter
from pymongo.errors import WriteError

In [3]:
def get_db(db_name="dungeons"):
    """Conecta al servidor MongoDB que se ejecuta en localhost en el
    puerto 27017, selecciona la base de datos dungeons y la devuelve."""
    client = MongoClient("localhost", 27017)
    db = client[db_name]
    return db


def run_query(filter, project, collection_name, db_name="dungeons"):
    """Ejecuta una query genérica en la colección collection_name de la
    base de datos db_name.

    Args:
        filter: El filtro para buscar documentos (equivalente a
            la cláusula WHERE en SQL).
        project: La proyección para seleccionar qué campos incluir
            (equivalente a SELECT en SQL).
        collection_name: El nombre de la colección en la que se
            ejecutará la query.
        db_name: El nombre de la base de datos en la que se ejecutará
            la query.

    Returns:
        Un diccionario o lista de diccionarios con los resultados de la query.
    """
    db = get_db(db_name)
    result = db[collection_name].find(filter, project)
    result_list = list(result)
    result_json = json.dumps(result_list)
    result_json = json.loads(result_json)
    db.client.close()
    return result_json


def run_aggregate(pipeline, collection_name, db_name="dungeons"):
    """Ejecuta una consulta de agregación en la colección collection_name
    de la base de datos db_name.

    Args:
        pipeline: La tubería de agregación para operaciones más complejas 
            (equivalente a una serie de operaciones en SQL).
        collection_name: El nombre de la colección en la que se ejecutará
            la consulta de agregación.
        db_name: El nombre de la base de datos en la que se ejecutará.
    """
    db = get_db(db_name)
    result = db[collection_name].aggregate(pipeline)
    result_list = list(result)
    result_json = json.dumps(result_list)
    result_json = json.loads(result_json)
    db.client.close()
    return result_json


def run_insert(data, collection_name, db_name="dungeons"):
    """Inserta un documento en la colección."""
    db = get_db(db_name)
    db[collection_name].insert_one(data)
    db.client.close()


def run_append(data, collection_name, query, array_name, db_name="dungeons"):
    """Inserta un documento en un array de un documento existente.
    
    Args:
        data: El documento a insertar en el array.
        collection_name: El nombre de la colección en la que se insertará
            el documento.
        query: El filtro para encontrar el documento en el que se insertará
            el nuevo documento.
        array_name: El nombre del array en el que se insertará el documento.
        db_name: El nombre de la base de datos en la que se insertará el
            documento.
    """
    db = get_db(db_name)
    update = [
        {
            "$set": {
                array_name: {
                    "$ifNull": [{"$concatArrays": [f"${array_name}", data]}, data]
                }
            }
        }
    ]
    db[collection_name].update_many(query, update)
    db.client.close()

#### GET /loot: Devuelve los siguientes campos de todos los objetos del juego: idL, name.

In [5]:
def get_loot_data():
    # Definir el filtro y la proyección
    filter = {}
    project = {"id": 1, "name": 1, "_id": 0}

    # Ejecutar la consulta
    result = run_query(filter, project, "Loot")
    return result


get_loot_data()

[{'id': 2, 'name': "Alchemist's Fire (flask)"},
 {'id': 8, 'name': 'Amulet'},
 {'id': 20, 'name': 'Baipes'},
 {'id': 47, 'name': 'Blunderbuss (Exandria)'},
 {'id': 49, 'name': 'Bomb'},
 {'id': 54, 'name': 'Breastplate'},
 {'id': 59, 'name': "Burglar's Pack"},
 {'id': 68, 'name': 'Carriage'},
 {'id': 69, 'name': 'Cart'},
 {'id': 75, 'name': 'Chain Mail'},
 {'id': 76, 'name': 'Chain Shirt'},
 {'id': 79, 'name': 'Chariot'},
 {'id': 98, 'name': 'Crossbow, Hand'},
 {'id': 100, 'name': 'Crossbow, Light'},
 {'id': 102, 'name': 'Crystal'},
 {'id': 107, 'name': "Diplomat's Pack"},
 {'id': 108, 'name': 'Disguise Kit'},
 {'id': 109, 'name': 'Dogsled'},
 {'id': 111, 'name': 'Double-Bladed Scimitar'},
 {'id': 120, 'name': "Dungeoneer's Pack"},
 {'id': 128, 'name': "Entertainer's Pack"},
 {'id': 130, 'name': "Explorer's Pack"},
 {'id': 142, 'name': 'Glaive'},
 {'id': 143, 'name': "Glassblower's Tools"},
 {'id': 146, 'name': 'Greataxe'},
 {'id': 152, 'name': 'Gunpowder, Keg'},
 {'id': 155, 'name': 'H

#### GET /loot/{loot_id}: Devuelve toda la información de un objeto del juego, incluidas las habitaciones donde aparece, de las habitaciones devuelve: idR, room_name y de la mazmorra a la que pertenece: idM, dungeon_name.

In [21]:
def get_loot_id(id):
    # Definir el filtro y la proyección
    filter = {"id": id}
    project = {
        "id": 1,
        "gold": 1,
        "name": 1,
        "type1": 1,
        "type2": 1,
        "weight": 1,
        "_id": 0,
        "in_rooms.room_id": 1,
        "in_rooms.room_name": 1,
        "in_rooms.dungeon_id": 1,
        "in_rooms.dungeon_name": 1,
    }

    # Ejecutar la consulta
    result = run_query(filter, project, "Loot")
    return result


get_loot_id(id=3)

[{'id': 3,
  'gold': 50.0,
  'name': "Alchemist's Supplies",
  'type1': 'Tool',
  'type2': 'Utility',
  'weight': '8',
  'in_rooms': [{'room_id': 441,
    'room_name': 'fierce hall ',
    'dungeon_id': 13,
    'dungeon_name': 'Greatcliffe, Castle of the Magnificent Sumo Wrestlers'}]}]

#### GET /monster: Devuelve los siguientes campos de todos los monstruos del juego: idM, name, level, type.

In [7]:
def get_monster():
    # Definir el filtro y la proyección
    filter = {}
    project = {"id": 1, "name": 1, "level": 1, "type": 1, "_id": 0}

    # Ejecutar la consulta
    result = run_query(filter, project, "Monster")
    return result


get_monster()

[{'id': 3, 'name': 'rakshasa', 'type': 'fiend', 'level': 100},
 {'id': 5, 'name': 'vampire', 'type': 'undead', 'level': 100},
 {'id': 7, 'name': 'blink dog', 'type': 'fey', 'level': 1},
 {'id': 8, 'name': 'acolyte', 'type': 'humanoid', 'level': 1},
 {'id': 10, 'name': 'commoner', 'type': 'humanoid', 'level': 1},
 {'id': 11, 'name': 'cultist', 'type': 'humanoid', 'level': 1},
 {'id': 12, 'name': 'guard', 'type': 'humanoid', 'level': 1},
 {'id': 15, 'name': 'flying sword', 'type': 'construct', 'level': 1},
 {'id': 18, 'name': 'beholder', 'type': 'aberration', 'level': 100},
 {'id': 19, 'name': 'death tyrant', 'type': 'undead', 'level': 100},
 {'id': 21, 'name': 'crawling claw', 'type': 'undead', 'level': 1},
 {'id': 22, 'name': 'death knight', 'type': 'undead', 'level': 100},
 {'id': 23, 'name': 'demilich', 'type': 'undead', 'level': 100},
 {'id': 29, 'name': 'clay golem', 'type': 'construct', 'level': 50},
 {'id': 31, 'name': 'iron golem', 'type': 'construct', 'level': 100},
 {'id': 32,

#### GET /monster/{monster_id}: Devuelve toda la información sobre un monstruo del juego, incluidas las habitaciones donde aparecen, de las habitaciones devuelve: idR, room_name y de la mazmorra a la que pertenece: idM, dungeon_name.

In [8]:
def get_monster_id(id):
    # Definir el filtro y la proyección
    filter = {"id": id}
    project = {
        "id": 1,
        "exp": 1,
        "name": 1,
        "type": 1,
        "level": 1,
        "place": 1,
        "man_page": 1,
        "_id": 0,
        "in_rooms.room_id": 1,
        "in_rooms.room_name": 1,
        "in_rooms.dungeon_id": 1,
        "in_rooms.dungeon_name": 1,
    }

    # Ejecutar la consulta
    result = run_query(filter, project, "Monster")
    return result


get_monster_id(id=3)

[{'id': 3,
  'exp': 10000,
  'name': 'rakshasa',
  'type': 'fiend',
  'level': 100,
  'place': 'city',
  'man_page': 257}]

#### GET /dungeon: Devuelve los siguientes campos de todas las mazmorras del juego: idD, name.

In [9]:
def get_dungeon():
    """Agrupa las habitaciones por dungeon_name y dungeon_id para obtener 
    todas las mazmorras."""
    # Definir el filtro y la proyección
    pipeline = [
        {
            "$group": {
                "_id": {"dungeon_name": "$dungeon_name", "dungeon_id": "$dungeon_id"}
            }
        },
        {"$replaceRoot": {"newRoot": "$_id"}},
    ]

    # Ejecutar la consulta
    result = run_aggregate(pipeline, "Rooms")
    return result


get_dungeon()

[{'dungeon_name': 'Schyppegarth, Laboratory of the Sticky Robots',
  'dungeon_id': 14},
 {'dungeon_name': 'Withystream, Castle of the Screeching Otaku',
  'dungeon_id': 4},
 {'dungeon_name': 'Garthwort, Mines of the Elderly Emperors',
  'dungeon_id': 18},
 {'dungeon_name': 'Shadysparth, Hobble of the Wandering Stoners',
  'dungeon_id': 2},
 {'dungeon_name': 'Burghap, Prison of the Jealous Hippies', 'dungeon_id': 0},
 {'dungeon_name': 'Marshgreat, Catacombs of the Wandering Degenerates',
  'dungeon_id': 17},
 {'dungeon_name': 'Dalhylles, Culverts of the Grumpy Bandits',
  'dungeon_id': 19},
 {'dungeon_name': 'Watergarth, Stockade of the Feminist Presidents',
  'dungeon_id': 7},
 {'dungeon_name': 'Corrieclock, Pit of the Sexy Unknowns', 'dungeon_id': 15},
 {'dungeon_name': 'Greatcliffe, Castle of the Magnificent Sumo Wrestlers',
  'dungeon_id': 13},
 {'dungeon_name': 'Burgstream, Culverts of the Bashful Sumo Wrestlers',
  'dungeon_id': 1},
 {'dungeon_name': 'Wanton, Culverts of the Jealo

#### GET /dungeon/{dungeon_id}: Devuelve información sobre una mazmorra del juego. Debe devolver: idM, name y lore. Además, este endpoint se utiliza para alimentar un grafo interactivo por lo que requiere la siguiente información:
1) El nombre e id de cada habitación de la mazmorra.
2) Las conexiones entre habitaciones de la mazmorra.
3) El id y el nombre de los monstruos que aparecen en cada habitación.
4) El id y el nombre de los tesoros que aparecen en cada habitación.
5) El número de comentarios de cada categoría que hay en cada habitación.

In [13]:
def get_dungeon_id(id):

    # Definir el filtro y la proyección
    pipeline_1 = [
        {"$match": {"dungeon_id": id}},
        {"$unwind": "$hints"},
        {"$match": {"hints.category": "lore"}},
        {
            "$group": {
                "_id": {"id": "$dungeon_id", "name": "$dungeon_name"},
                "lore": {"$push": {"text": "$hints.hintText",
                                   "creation_date": "$hints.creation_date",
                                   "publish_by": "$hints.publish_by.email"}},
            }
        },
    ]

    # Ejecutar la consulta
    result_1 = run_aggregate(pipeline_1, "Rooms")

    # Realizmos una segunda consulta para
    pipeline_2 = [
        {"$match": {"dungeon_id": id}},
        {
            "$group": {
                "_id": "$dungeon_name",
                "data": {
                    "$push": {
                        "room_id": "$room_id",
                        "name": "$room_name",
                        "Monster": "$monsters",
                        "Loot": "$loot",
                        "n_size": {"$size": {"$ifNull": ["$hints", []]}},
                        "room_connections": "$rooms_connected",
                    }
                },
            }
        },
    ]

    # Ejecutar la consulta
    result_2 = run_aggregate(pipeline_2, "Rooms")

    result_1[0]["data"] = result_2[0]["data"]
    return result_1


get_dungeon_id(id=0)

[{'_id': {'id': 0, 'name': 'Burghap, Prison of the Jealous Hippies'},
  'lore': [{'text': 'Граница пропаганда банк домашний роскошный. Неожиданно обида уточнить прощение. Вообще намерение палец рассуждение задержать дьявол. Советовать еврейский увеличиваться ботинок опасность угодный.\\nУточнить пища народ бетонный избегать эпоха возникновение плясать. Новый увеличиваться полюбить природа второй правление. Расстройство ягода рай выгнать поколение мягкий решение.',
    'creation_date': '2022-03-21 06:01:50.000000',
    'publish_by': 'rodion_2002@example.net'},
   {'text': 'Die Sonne Licht schnell zeigen Geburtstag kennen. Las darin hängen wichtig Vogel heiß vom nimmt. Fröhlich Abend Teller rechnen. Grün einfach fast.\\nSelbst haben hören wirklich. Verstecken Zahl sitzen Hund schwimmen baden lange. Weil gleich nichts Stunde. Bin Freund gleich zum essen.\\nWeiß dumm doch einfach Lehrer Mann kommen. Müssen Glück Leute heute los gut wird.\\nGar steigen Wetter gehören Frage um Wagen tragen. 

#### GET /room/{room_id}: Devuelve la siguiente información de una habitación: idR, name, inWP y outWP. Además, incluye la siguiente información de todos los monstruos incluidos en la habitación (idM, name, type, level, place, exp, manPage) y de los tesoros (idL, name, type1, type2, weight, gold). Por último, incluye todos los comentarios que se hayan realizado para esa habitación, cada comentario debe incluir: email y userName, country, creationDate del usuario que lo realizo y texto, fecha de publicación y categoría del comentario.

In [22]:
def get_room_id(id):
    # Definir el filtro y la proyección
    filter = {"room_id": id}
    project = {
        "room_id": 1,
        "room_name": 1,
        "in_waypoint": 1,
        "out_waypoint": 1,
        "Loot": 1,
        "Monster": 1,
        "hints": 1,
        "_id": 0,
    }

    # Ejecutar la consulta
    result = run_query(filter, project, "Rooms")
    return result


get_room_id(id=50)

[{'hints': [{'category': 'lore',
    'hintText': 'Похороны постоянный эффект изображать передо. Цвет господь порядок триста миф.\\nЛететь заработать хотеть нож социалистический. Умолять материя предоставить написать деньги факультет мусор.\\nНекоторый полевой невозможно интеллектуальный тесно зато. Плавно отражение мрачно выбирать.\\nНож пропасть тревога штаб выраженный. Мимо темнеть легко желание.\\nГосподь правый госпожа равнодушный. Военный палата необычный. Светило проход аж изучить деловой командир.',
    'publish_by': {'email': 'fortunat2020@example.com',
     'country': 'ru_RU',
     'user_name': 'velimirmolchanov',
     'creation_date': '2021-07-19'},
    'creation_date': '2022-11-23 16:53:50.000000'},
   {'category': 'hint',
    'hintText': 'Adipisci sequi debitis eius vel iure. Iure tempora sed nesciunt.\\nSaepe ut aspernatur beatae possimus accusamus cum accusamus. In voluptatem alias beatae.\\nNon tenetur libero eos eligendi consequuntur atque commodi. Non facilis tempore a

#### GET /user: Devuelve los campos email, username y country de todos los usuarios.


In [12]:
def get_user():
    # Definir el filtro y la proyección
    filter = {}
    project = {
        "_id": 0,
        "email": 1,
        "username": 1,
        "country": 1,
    }

    # Ejecutar la consulta
    result = run_query(filter, project, "Users")
    return result


get_user()

[{'email': 'aaoki@example.com', 'country': 'ja_JP'},
 {'email': 'abaldwin@example.org', 'country': 'en_US'},
 {'email': 'abarbosa@example.org', 'country': 'pt_BR'},
 {'email': 'abaresi@example.com', 'country': 'it_IT'},
 {'email': 'abemikako@example.net', 'country': 'ja_JP'},
 {'email': 'abeminoru@example.org', 'country': 'ja_JP'},
 {'email': 'abeshohei@example.net', 'country': 'ja_JP'},
 {'email': 'abiagiotti@example.net', 'country': 'it_IT'},
 {'email': 'abril85@example.net', 'country': 'es_ES'},
 {'email': 'acastro@example.net', 'country': 'en_US'},
 {'email': 'acosta@example.org', 'country': 'pt_BR'},
 {'email': 'adalbertdehmel@example.com', 'country': 'de_DE'},
 {'email': 'adamellis@example.org', 'country': 'en_US'},
 {'email': 'adancepeda@example.org', 'country': 'es_ES'},
 {'email': 'adanherranz@example.org', 'country': 'es_ES'},
 {'email': 'adele81@example.org', 'country': 'fr_FR'},
 {'email': 'adeleimbert@example.com', 'country': 'fr_FR'},
 {'email': 'adelmofermi@example.org',

#### GET /user/{email}: Devuelve todos los campos de un usuario. Además, incluye todos los comentarios que ha realizado. De cada comentario incluye, el texto, la fecha de creación, la categoría, el id (Room.IdR) y nombre (Room.name) de la habitación a la que hace referencia el comentario, el id(Dungeon.IdD) y nombre(Dungeon.name) de la mazmorra donde está la habitación.


In [8]:
def get_user_id(email):
    # Definir el filtro y la proyección
    filter = {"email": email}
    project = {
        "_id": 0,
        "email": 1,
        "username": 1,
        "country": 1,
        "hints": 1,
    }

    # Ejecutar la consulta
    result = run_query(filter, project, "Users")
    return result


get_user_id(email="aaron03@example.com")

[{'email': 'aaron03@example.com',
  'hints': [{'text': 'Write already region help. Often blue board century.\\nSource important line happy him. As role firm.\\nDog natural five available high thousand. Student when phone help.\\nNice campaign produce identify. Tree particular image how fight. National now look center trip possible board.\\nFriend beautiful sign must plan evening answer. Federal result anything determine decide mind.\\nShe activity want join. There product daughter development.\\nWin send gun pull short.\\nHospital service person theory heart law c',
    'category': 'lore',
    'creation_date': '2019-02-05 03:15:54.000000',
    'referemces_room': {'room_id': 296,
     'room_name': 'storage room of kings',
     'dungeon_id': 7,
     'dungeon_name': 'Watergarth, Stockade of the Feminist Presidents'}},
   {'text': 'Pick sign concern laugh give thus police. Author truth person new road number.\\nSense leader decade traditional few. Instead final thought allow ok environment

#### POST /comment: Añade un nuevo comentario. Necesita de autenticación de usuario.
- ##### Parametros: user_email (str), room_id (int), text (str), category (str).


In [9]:
def post_comment(email, room_id, text, category):
    # Dado el id de una habitación, obtener room_name, dungeon_id y dungeon_name
    filter_room = {"room_id": room_id}
    project_room = {"room_name": 1, "dungeon_id": 1, "dungeon_name": 1, "_id": 0}
    room = run_query(filter_room, project_room, "Rooms")
    room_name = room[0]["room_name"]
    dungeon_id = room[0]["dungeon_id"]
    dungeon_name = room[0]["dungeon_name"]
 
    # Dado el email de un usuario, obtener user_name, country y creation_date
    filter_email = {"email": email}
    project_email = {"user_name": 1, "country": 1, "creation_date": 1, "_id": 0}
    user = run_query(filter_email, project_email, "Users")
    user_name = user[0]["user_name"]
    country = user[0]["country"]
    creation_date = user[0]["creation_date"]

    # Crear los docuemntos a insertar
    created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    comment_room = {
        "category": category,
        "hintText": text,
        "published_by": {
            "email": email,
            "country": country,
            "user_name": user_name,
            "creation_date": creation_date,
        },
        "creation_date": created_at,
    }

    comment_user = {
        "text": text,
        "category": category,
        "creation_date": created_at,
        "referemces_room": {
            "room_id": room_id,
            "room_name": room_name,
            "dungeon_id": dungeon_id,
            "dungeon_name": dungeon_name,
        },
    }

    # Insertar los documentos
    run_append([comment_room], "Rooms", filter_room, "hints")
    run_append([comment_user], "Users", filter_email, "hints")


post_comment("aaoki@example.com", 5, "This is a test comment", "lore")

#### POST /monster: Añade un nuevo monstruo al juego. Necesita de autenticación de admin.
- ##### Parámetros en cabecera: name (str), type (str), level (int), place (str), exp (float), manPage (int).


In [15]:
def post_monster(id, name, type, level, place, exp, man_page):
    data = {
        "id": id,
        "exp": exp,
        "name": name,
        "type": type,
        "level": level,
        "place": place,
        "in_rooms": None,
        "man_page": man_page,
    }
    run_insert(data, "Monster")


post_monster(266, "Goblin", "Humanoid", 1, "Dungeon", 100, 2)

In [16]:
def post_monster(name, type, level, place, exp, man_page):
    # Get the next id using max aggregation
    pipeline = [{"$group": {"_id": None, "max_id": {"$max": "$id"}}}]
    result = run_aggregate(pipeline, "Monster")

    monster_id = result[0]["max_id"] + 1

    data = {
        "id": monster_id,
        "exp": exp,
        "name": name,
        "type": type,
        "level": level,
        "place": place,
        "in_rooms": None,
        "man_page": man_page,
    }
    run_insert(data, "Monster")
    print(f"Añadido el monstruo {name} con id {monster_id}")


post_monster("Goblin Plus", "Humanoid", 2, "Dungeon", 110, 2)

Añadido el monstruo Goblin Plus con id 267


#### POST /loot: Añade un nuevo objeto al juego. Necesita de autenticación de admin.
- ##### Parámetros en cabecera: name (str), type1 (str), type2 (str), weight (str), gold (float).


In [17]:
def post_loot(id, name, type1, type2, weight, gold):
    data = {
        "id": id,
        "gold": gold,
        "name": name,
        "type1": type1,
        "type2": type2,
        "weight": weight,
        "in_rooms": None,
    }
    run_insert(data, "Loot")


post_loot(383, "Platinum coin", "Coin", "Platinum", 0.1, 1000)

In [18]:
def post_loot(name, type1, type2, weight, gold):
    # Get the next id using max aggregation
    pipeline = [{"$group": {"_id": None, "max_id": {"$max": "$id"}}}]
    result = run_aggregate(pipeline, "Loot")

    loot_id = result[0]["max_id"] + 1

    data = {
        "id": loot_id,
        "gold": gold,
        "name": name,
        "type1": type1,
        "type2": type2,
        "weight": weight,
        "in_rooms": None,
    }
    run_insert(data, "Loot")
    print(f"Añadido el loot {name} con id {loot_id}")


post_loot("Platinum coin Plus", "Coin", "Platinum", 0.1, 1000)

Añadido el loot Platinum coin Plus con id 384


#### POST /room: Añade una nueva habitación a una mazmorra. Necesita de autenticación de admin.
- ##### Parámetros de cabecera: dungeon_id (int), dungeon_name(str), dungeon_lore(str), room_name (str), rooms_connected (list[int]), [inWP(str)], [outWP(str)].


In [19]:
def post_room(
    room_id,
    room_name,
    dungeon_id,
    dungeon_name,
    rooms_connected,
    in_waypoint=None,
    out_waypoint=None,
):
    filter_connections = {"room_id": {"$in": rooms_connected}}

    project_connections = {"room_id": 1, "room_name": 1, "_id": 0}

    rooms = run_query(filter_connections, project_connections, "Rooms")

    data = {
        "Loot": None,
        "hints": None,
        "room_id": room_id,
        "monsters": None,
        "room_name": room_name,
        "dungeon_id": dungeon_id,
        "in_waypoint": in_waypoint,
        "dungeon_name": dungeon_name,
        "out_waypoint": out_waypoint,
        "rooms_connected": rooms,
    }
    run_insert(data, "Rooms")


post_room(
    719,
    "test_room",
    19,
    "Dalhylles, Culverts of the Grumpy Bandits",
    [718],
    in_waypoint=718,
    out_waypoint=None,
)

In [20]:
def post_room(
    room_name,
    dungeon_id,
    dungeon_name,
    rooms_connected,
    in_waypoint=None,
    out_waypoint=None,
):
    # Get the next id using max aggregation
    pipeline = [{"$group": {"_id": None, "max_id": {"$max": "$room_id"}}}]
    result = run_aggregate(pipeline, "Rooms")

    room_id = result[0]["max_id"] + 1

    filter_connections = {"room_id": {"$in": rooms_connected}}

    project_connections = {"room_id": 1, "room_name": 1, "_id": 0}

    rooms = run_query(filter_connections, project_connections, "Rooms")

    data = {
        "Loot": None,
        "hints": None,
        "room_id": room_id,
        "monsters": None,
        "room_name": room_name,
        "dungeon_id": dungeon_id,
        "in_waypoint": in_waypoint,
        "dungeon_name": dungeon_name,
        "out_waypoint": out_waypoint,
        "rooms_connected": rooms,
    }
    run_insert(data, "Rooms")

    run_append(
        [{"room_id": room_id, "room_name": room_name}],
        "Rooms",
        filter_connections,
        "rooms_connected",
    )

    print(f"Añadida la habitación {room_name} con id {room_id}")


post_room(
    "test_room",
    19,
    "Dalhylles, Culverts of the Grumpy Bandits",
    [719],
    in_waypoint=719,
    out_waypoint=None,
)

Añadida la habitación test_room con id 720


#### PUT /room/{room_id}/monster: Sobrescribe los monstruos de una sala de una mazmorra, los monstruo debe de existir.
- ##### Parámetros cabecera: monsters(list[int])


In [21]:
def put_room_monsters(room_id, monster):
    # First, check if all the monsters exist
    filter_monster = {"id": {"$in": monster}}
    project_monster = {
        "id": 1,
        "exp": 1,
        "name": 1,
        "type": 1,
        "level": 1,
        "place": 1,
        "man_page": 1,
        "_id": 0,
    }
    result_monster = run_query(filter_monster, project_monster, "Monster")
    if len(result_monster) != len(set(monster)):
        raise ValueError("One or more monsters do not exist")

    # Duplicate the monster documents that are duplicated in monster
    # Use Counter to count occurrences in monster
    monster_counter = dict(Counter(monster))

    # Iterate over the items and counts in the monster_counter dictionary
    for item, count in monster_counter.items():
        if count > 1:
            # Get the data of the monster from the result_monster list
            monster_data = [
                monster for monster in result_monster if monster["id"] == item
            ][0]
            # Duplicate the monster_data count - 1 times
            result_monster.extend([monster_data] * (count - 1))

    # Now, get the current monster
    filter_room = {"room_id": room_id}
    project_room_monster = {"monsters.id": 1, "_id": 0}

    # Append an empty array to the monster collection, since $pull does not work with null values
    run_append([], "Rooms", filter_room, "monsters")
    monster_in_room = run_query(filter_room, project_room_monster, "Rooms")
    old_monster_ids = [monster["id"] for monster in monster_in_room[0]["monsters"]]

    project_room = {
        "room_id": 1,
        "room_name": 1,
        "dungeon_id": 1,
        "dungeon_name": 1,
        "_id": 0,
    }
    room = run_query(filter_room, project_room, "Rooms")[0]

    # Now, delete the old monster by deleting the documents in the monster array in the old monster ids with the room_id

    filter_old_monster_room = {"id": {"$in": old_monster_ids}}
    update_old_monster_room = {"$pull": {"in_rooms": {"room_id": room_id}}}
    db = get_db()

    try:
        db["Monster"].update_many(filter_old_monster_room, update_old_monster_room)
    except WriteError:
        # Add an empty array to the monster collection, since $pull does not work with null values
        run_append([], "Monster", filter_old_monster_room, "in_rooms")
        db["Monster"].update_many(filter_old_monster_room, update_old_monster_room)

    db.client.close()

    # Now, add the new monster to the monster collection
    for monster_id, amount in monster_counter.items():
        in_rooms = {"amount": amount, **room}
        filter_monster_each = {"id": monster_id}
        run_append([in_rooms], "Monster", filter_monster_each, "in_rooms")

    # Set the monster array to null
    db = get_db()
    db["Rooms"].update_one(filter_room, {"$set": {"monsters": None}})

    # Add the new monster to the room collection
    run_append(result_monster, "Rooms", filter_room, "monsters")


put_room_monsters(719, [266, 267, 267])

#### PUT /room/{room_id}/loot: Sobrescribe los tesoros de una sala de una mazmorra, los tesoros deben de existir.
- ##### Parámetros cabecera: loot(list[int])


In [22]:
def put_room_loot(room_id, loot):
    # First, check if all the loots exist
    filter_loot = {"id": {"$in": loot}}
    project_loot = {
        "id": 1,
        "gold": 1,
        "name": 1,
        "type1": 1,
        "type2": 1,
        "weight": 1,
        "_id": 0,
    }
    result_loot = run_query(filter_loot, project_loot, "Loot")
    if len(result_loot) != len(set(loot)):
        raise ValueError("One or more loots do not exist")

    # Duplicate the loot documents that are duplicated in loot
    # Use Counter to count occurrences in loot
    loot_counter = dict(Counter(loot))

    # Iterate over the items and counts in the loot_counter dictionary
    for item, count in loot_counter.items():
        if count > 1:
            # Get the data of the loot from the result_loot list
            loot_data = [loot for loot in result_loot if loot["id"] == item][0]
            # Duplicate the loot_data count - 1 times
            result_loot.extend([loot_data] * (count - 1))

    # Now, get the current loot
    filter_room = {"room_id": room_id}
    project_room_loot = {"loot.id": 1, "_id": 0}

    # Adds empty array to loot if it is null
    run_append([], "Rooms", filter_room, "Loot")
    loot_in_room = run_query(filter_room, project_room_loot, "Rooms")
    old_loot_ids = [loot["id"] for loot in loot_in_room[0]["Loot"]]

    project_room = {
        "room_id": 1,
        "room_name": 1,
        "dungeon_id": 1,
        "dungeon_name": 1,
        "_id": 0,
    }
    room = run_query(filter_room, project_room, "Rooms")[0]

    # Now, delete the old loot by deleting the documents in the loot array in the old loot ids with the room_id

    filter_old_loot_room = {"id": {"$in": old_loot_ids}}
    update_old_loot_room = {"$pull": {"in_rooms": {"room_id": room_id}}}
    db = get_db()

    try:
        db["Loot"].update_many(filter_old_loot_room, update_old_loot_room)
    except WriteError:
        # Add an empty array to the loot collection, since $pull does not work with null values
        run_append([], "Loot", filter_old_loot_room, "in_rooms")
        db["Loot"].update_many(filter_old_loot_room, update_old_loot_room)

    db.client.close()

    # Now, add the new loot to the loot collection
    for loot_id, amount in loot_counter.items():
        in_rooms = {"amount": amount, **room}
        filter_loot_each = {"id": loot_id}
        run_append([in_rooms], "Loot", filter_loot_each, "in_rooms")

    # Set the loot array to null
    db = get_db()
    db["Rooms"].update_one(filter_room, {"$set": {"Loot": None}})

    # Add the new loot to the room collection
    run_append(result_loot, "Rooms", filter_room, "Loot")


put_room_loot(719, [383, 384, 384])

#### PUT /room/{room_id}/connections: Sobrescribe las conexiones de una habitación.
- ##### Parámetros de cabecera: connections (list[int])


In [23]:
def put_room_connecions(room_id, rooms_connected):
    # First, check if all the rooms exist
    filter_rooms = {"room_id": {"$in": rooms_connected}}
    project = {"room_id": 1, "room_name": 1, "_id": 0}
    result_rooms = run_query(filter_rooms, project, "Rooms")
    if len(result_rooms) != len(rooms_connected):
        raise ValueError("One or more rooms do not exist")

    # Now, get the current connections
    filter_room = {"room_id": room_id}
    project = {"rooms_connected": 1, "_id": 0}

    current_connections = run_query(filter_room, project, "Rooms")[0]["rooms_connected"]

    current_ids = [room["room_id"] for room in current_connections]

    # Rooms deleted
    rooms_deleted = list(set(current_ids) - set(rooms_connected))

    # Rooms added
    rooms_added = list(set(rooms_connected) - set(current_ids))

    # From rooms_deleted, delete room_id from rooms_connected
    filter_removed = {"room_id": {"$in": rooms_deleted}}
    update = {"$pull": {"rooms_connected": {"room_id": room_id}}}

    # Get the name from the current room
    filter_current = {"room_id": room_id}
    project_current = {"room_name": 1, "_id": 0}
    current_connections = run_query(filter_current, project_current, "Rooms")

    current_name = current_connections[0]["room_name"]

    db = get_db()
    db["Rooms"].update_many(filter_removed, update)
    db.client.close()

    # From rooms_added, add room_id to rooms_connected
    filter_added = {"room_id": {"$in": rooms_added}}
    run_append(
        [{"room_id": room_id, "room_name": current_name}],
        "Rooms",
        filter_added,
        "rooms_connected",
    )

    # Set the rooms_connected array to null
    db = get_db()
    db["Rooms"].update_one(filter_room, {"$set": {"rooms_connected": None}})
    db.client.close()

    # Add the new connections
    run_append(result_rooms, "Rooms", filter_room, "rooms_connected")


put_room_connecions(719, [718, 720])

#### DELETE /room/{room_id}/: Borra una habitación y todos los comentarios asociados a ella.


In [24]:
def delete_room(room_id):
    # Definir el filtro
    filter_room = {"room_id": room_id}

    # Ejecutar la consulta
    db = get_db()
    db["Rooms"].delete_one(filter_room)

    # Buscar, en todos los usuarios, los hints que hacen referencia a la habitación eliminada y eliminarlos
    filter_users = {"hints.referemces_room.room_id": room_id}
    update_users = {"$pull": {"hints": {"referemces_room.room_id": room_id}}}

    db["Users"].update_many(filter_users, update_users)

    # Buscar en todos los monstruos, los in_rooms que hacen referencia a la habitación eliminada y eliminarlos
    filter_monsters = {"in_rooms.room_id": room_id}
    update_monsters = {"$pull": {"in_rooms": {"room_id": room_id}}}

    db["Monster"].update_many(filter_monsters, update_monsters)

    # Buscar en todos los loots, los in_rooms que hacen referencia a la habitación eliminada y eliminarlos
    filter_loots = {"in_rooms.room_id": room_id}
    update_loots = {"$pull": {"in_rooms": {"room_id": room_id}}}
    db["Loot"].update_many(filter_loots, update_loots)

    db.client.close()


delete_room(720)

#### DELETE /monster/{monster_id}/: Borra un monstruo.


In [25]:
def delete_monster(id):
    # Definir el filtro
    filter_monster = {"id": id}

    # Ejecutar la consulta
    db = get_db()
    db["Monster"].delete_one(filter_monster)

    filter_room = {"monsters.id": id}

    # En todas las rooms, eliminar el monster con id
    update = {"$pull": {"monsters": {"id": id}}}
    db["Rooms"].update_many(filter_room, update)

    db.client.close()


delete_monster(266)

#### DELETE /loot/{loot_id}/: Borra un tesoro.

In [26]:
def delete_loot(id):
    # Definir el filtro
    filter_loot = {"id": id}

    # Ejecutar la consulta
    db = get_db()
    db["Loot"].delete_one(filter_loot)

    filter_room = {"loot.id": id}

    # En todas las rooms, eliminar el loot con id
    update = {"$pull": {"Loot": {"id": id}}}
    db["Rooms"].update_many(filter_room, update)

    db.client.close()


delete_loot(383)